<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Train Practice

## *Data Science Unit 4 Sprint 2 Assignment 4*

Continue to use TensorFlow Keras & a sample of the [Quickdraw dataset](https://github.com/googlecreativelab/quickdraw-dataset) to build a sketch classification model. The dataset has been sampled to only 10 classes and 10000 observations per class. Apply regularization techniques to your model. 

*Don't forgot to switch to GPU on Colab!*

## Regularization

Using your best performing model from the previous module, apply each of the following regularization strategies: 
* Early Stopping
* Dropout
* Weight Decay
* Weight Constraint


In [22]:
!pip install wandb -q
!pip install opencv-python
import wandb
from wandb.keras import WandbCallback
%load_ext tensorboard

import os
import seaborn as sns

import numpy as np
import pandas as pd

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn import model_selection
from sklearn.model_selection import train_test_split, learning_curve, KFold, cross_val_score, StratifiedKFold

import keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from tensorflow.keras import layers
from tensorflow.keras.layers import Flatten, Dense, Conv2D, MaxPooling2D, Dropout, Flatten
from tensorflow.keras.layers import ReLU
from keras.utils import np_utils
from keras.optimizers import SGD
from keras.optimizers import RMSprop, SGD, Adam, Nadam
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, Callback, EarlyStopping
import tensorflow as tf
from tensorflow import keras

import IPython
# Install and import the Keras Tuner.
!pip install -U keras-tuner
import kerastuner as kt

import os
import itertools
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import AxesGrid
import random
random.seed(42)
%matplotlib inline
!pip install --upgrade wandb
!wandb login 351c7f296c88cd03681b3eaa2b3400ba29ad130b
wandb_project = 'ass-4'

# Ignore excessive warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

URL_ = "https://github.com/LambdaSchool/DS-Unit-4-Sprint-2-Neural-Networks/blob/main/quickdraw10.npz?raw=true"

path_to_zip = tf.keras.utils.get_file('./quickdraw10.npz', origin=URL_, extract=False)
data = np.load(path_to_zip)

X = data['arr_0']
y = data['arr_1']

print(X.shape)
print(y.shape)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.40)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

class_names = ['apple',
 'anvil',
 'airplane',
 'banana',
 'The Eiffel Tower',
 'The Mona Lisa',
 'The Great Wall of China',
 'alarm clock',
 'ant',
 'asparagus']

img_width=28
img_height=28

X_train = X_train.astype('float32') / 255.
X_test = X_test.astype('float32') / 255.

# # reshape input data
# X_train = X_train.reshape(X_train.shape[0], img_width, img_height, 1)
# X_test = X_test.reshape(X_test.shape[0], img_width, img_height, 1)

# # one hot encode outputs
# y_train = np_utils.to_categorical(y_train)
# y_test = np_utils.to_categorical(y_test)
# num_classes = y_test.shape[1]
!pip install keras-tuner

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
Requirement already up-to-date: keras-tuner in /opt/anaconda3/envs/U4-S1-NLP/lib/python3.8/site-packages (1.0.1)
Requirement already up-to-date: wandb in /opt/anaconda3/envs/U4-S1-NLP/lib/python3.8/site-packages (0.10.11)
wandb: Appending key for api.wandb.ai to your netrc file: /Users/codyreece/.netrc
(100000, 784)
(100000,)
(60000, 784) (40000, 784) (60000,) (40000,)


In [23]:
def tune_optimizer_model(hp):
    logdir = os.path.join("logs", "EarlyStopping-Loss")

    tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
    stop = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=3)

    model = tf.keras.Sequential([
        Flatten(input_shape=(28,28)),
        Dense(128),
        ReLU(negative_slope=.01),
        Dense(128),
        ReLU(negative_slope=.01),
        Dense(128),
        ReLU(negative_slope=.01),
        Dense(10, activation='softmax')
    ])

    optimizer = hp.Choice('optimizer', ['adam', 'sgd', 'rmsprop', 'nadam'])

    model.compile(
            optimizer=optimizer,
            loss = 'sparse_categorical_crossentropy', 
            metrics = ['accuracy'])
    return model

In [24]:
from pprint import pprint
from kerastuner.tuners import RandomSearch, BayesianOptimization

MAX_TRIALS = 2
EXECUTIONS_PER_TRIAL = 2
RANDOM_SEED=42

tuner = RandomSearch(
    tune_optimizer_model,
    objective='val_accuracy',
    max_trials=MAX_TRIALS,
    executions_per_trial=EXECUTIONS_PER_TRIAL,
    directory='test_dir', 
    project_name='tune_optimizer',
    seed=RANDOM_SEED
)

tuner.search_space_summary()

In [25]:
TRAIN_EPOCHS = 20

tuner.search(x=X_train,
             y=y_train,
             epochs=TRAIN_EPOCHS,
             validation_data=(X_test, y_test))

In [26]:
tuner.results_summary()

In [27]:
tuner.oracle.get_best_trials(num_trials=1)[0].hyperparameters.values

{'optimizer': 'adam'}

In [28]:
best_model = tuner.get_best_models()[0]

In [29]:
best_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               100480    
_________________________________________________________________
re_lu (ReLU)                 (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
re_lu_1 (ReLU)               (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
re_lu_2 (ReLU)               (None, 128)               0

In [42]:
def tune_rl_momentum_model(hp):
    model = keras.Sequential()
    model.add(keras.layers.Dense(units=18, activation="relu", input_shape=[X_train.shape[1]]))

    model.add(keras.layers.Dense(10, activation='sigmoid'))

    lr = hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])
    momentum = hp.Choice('momentum', [0.0, 0.2, 0.4, 0.6, 0.8, 0.9])

    model.compile(
        optimizer=keras.optimizers.SGD(lr, momentum=momentum),
        loss = 'sparse_categorical_crossentropy', 
        metrics = ['accuracy'])
    return model

In [43]:
def create_random_tuner(model_builder, project_name):
  tuner = RandomSearch(
    model_builder,
    objective='val_accuracy',
    max_trials=MAX_TRIALS,
    executions_per_trial=EXECUTIONS_PER_TRIAL,
    directory='test_dir', 
    project_name=project_name,
    seed=RANDOM_SEED
  )

  tuner.search_space_summary()
  return tuner

In [44]:
def random_search_params(model_builder, project_name):
  tuner = create_random_tuner(model_builder, project_name)
  tuner.search(x=X_train,
             y=y_train,
             epochs=TRAIN_EPOCHS,
             validation_data=(X_test, y_test))
  
  tuner.results_summary()

  pprint(tuner.oracle.get_best_trials(num_trials=1)[0].hyperparameters.values)

In [45]:
random_search_params(tune_rl_momentum_model, project_name="tune_lr_momentum")

Epoch 1/20
1875/1875 [==============================] - ETA: 0s - loss: 2.3381 - accuracy: 0.12 - ETA: 1s - loss: 2.3122 - accuracy: 0.09 - ETA: 1s - loss: 2.3100 - accuracy: 0.09 - ETA: 1s - loss: 2.3098 - accuracy: 0.09 - ETA: 0s - loss: 2.3099 - accuracy: 0.09 - ETA: 0s - loss: 2.3088 - accuracy: 0.09 - ETA: 0s - loss: 2.3094 - accuracy: 0.09 - ETA: 0s - loss: 2.3092 - accuracy: 0.09 - ETA: 0s - loss: 2.3088 - accuracy: 0.09 - ETA: 0s - loss: 2.3089 - accuracy: 0.09 - ETA: 0s - loss: 2.3085 - accuracy: 0.09 - ETA: 0s - loss: 2.3079 - accuracy: 0.09 - ETA: 0s - loss: 2.3070 - accuracy: 0.09 - ETA: 0s - loss: 2.3066 - accuracy: 0.09 - ETA: 0s - loss: 2.3054 - accuracy: 0.10 - ETA: 0s - loss: 2.3047 - accuracy: 0.10 - ETA: 0s - loss: 2.3041 - accuracy: 0.10 - ETA: 0s - loss: 2.3040 - accuracy: 0.10 - ETA: 0s - loss: 2.3034 - accuracy: 0.10 - ETA: 0s - loss: 2.3031 - accuracy: 0.10 - ETA: 0s - loss: 2.3024 - accuracy: 0.10 - ETA: 0s - loss: 2.3017 - accuracy: 0.10 - ETA: 0s - loss: 2.30

Epoch 14/20
1875/1875 [==============================] - ETA: 0s - loss: 1.9908 - accuracy: 0.37 - ETA: 1s - loss: 2.0536 - accuracy: 0.30 - ETA: 1s - loss: 2.0588 - accuracy: 0.30 - ETA: 1s - loss: 2.0583 - accuracy: 0.30 - ETA: 0s - loss: 2.0595 - accuracy: 0.30 - ETA: 0s - loss: 2.0564 - accuracy: 0.30 - ETA: 0s - loss: 2.0546 - accuracy: 0.31 - ETA: 0s - loss: 2.0543 - accuracy: 0.31 - ETA: 0s - loss: 2.0524 - accuracy: 0.31 - ETA: 0s - loss: 2.0521 - accuracy: 0.31 - ETA: 0s - loss: 2.0521 - accuracy: 0.31 - ETA: 0s - loss: 2.0517 - accuracy: 0.31 - ETA: 0s - loss: 2.0510 - accuracy: 0.31 - ETA: 0s - loss: 2.0506 - accuracy: 0.31 - ETA: 0s - loss: 2.0506 - accuracy: 0.31 - ETA: 0s - loss: 2.0497 - accuracy: 0.31 - ETA: 0s - loss: 2.0489 - accuracy: 0.31 - ETA: 0s - loss: 2.0485 - accuracy: 0.31 - ETA: 0s - loss: 2.0484 - accuracy: 0.31 - ETA: 0s - loss: 2.0483 - accuracy: 0.31 - ETA: 0s - loss: 2.0478 - accuracy: 0.31 - ETA: 0s - loss: 2.0474 - accuracy: 0.31 - ETA: 0s - loss: 2.0

1875/1875 [==============================] - ETA: 0s - loss: 2.2678 - accuracy: 0.12 - ETA: 1s - loss: 2.2583 - accuracy: 0.15 - ETA: 1s - loss: 2.2580 - accuracy: 0.15 - ETA: 1s - loss: 2.2577 - accuracy: 0.16 - ETA: 1s - loss: 2.2576 - accuracy: 0.15 - ETA: 1s - loss: 2.2561 - accuracy: 0.16 - ETA: 1s - loss: 2.2567 - accuracy: 0.16 - ETA: 1s - loss: 2.2561 - accuracy: 0.16 - ETA: 0s - loss: 2.2564 - accuracy: 0.16 - ETA: 0s - loss: 2.2560 - accuracy: 0.16 - ETA: 0s - loss: 2.2555 - accuracy: 0.16 - ETA: 0s - loss: 2.2548 - accuracy: 0.16 - ETA: 0s - loss: 2.2550 - accuracy: 0.16 - ETA: 0s - loss: 2.2553 - accuracy: 0.16 - ETA: 0s - loss: 2.2554 - accuracy: 0.16 - ETA: 0s - loss: 2.2554 - accuracy: 0.16 - ETA: 0s - loss: 2.2551 - accuracy: 0.16 - ETA: 0s - loss: 2.2552 - accuracy: 0.16 - ETA: 0s - loss: 2.2551 - accuracy: 0.16 - ETA: 0s - loss: 2.2546 - accuracy: 0.16 - ETA: 0s - loss: 2.2546 - accuracy: 0.16 - ETA: 0s - loss: 2.2543 - accuracy: 0.16 - ETA: 0s - loss: 2.2542 - accura

1875/1875 [==============================] - ETA: 0s - loss: 2.0733 - accuracy: 0.28 - ETA: 1s - loss: 2.1242 - accuracy: 0.23 - ETA: 1s - loss: 2.1252 - accuracy: 0.23 - ETA: 1s - loss: 2.1262 - accuracy: 0.23 - ETA: 1s - loss: 2.1265 - accuracy: 0.23 - ETA: 1s - loss: 2.1267 - accuracy: 0.23 - ETA: 0s - loss: 2.1267 - accuracy: 0.23 - ETA: 0s - loss: 2.1268 - accuracy: 0.23 - ETA: 0s - loss: 2.1270 - accuracy: 0.23 - ETA: 0s - loss: 2.1265 - accuracy: 0.23 - ETA: 0s - loss: 2.1264 - accuracy: 0.23 - ETA: 0s - loss: 2.1263 - accuracy: 0.23 - ETA: 0s - loss: 2.1258 - accuracy: 0.23 - ETA: 0s - loss: 2.1256 - accuracy: 0.23 - ETA: 0s - loss: 2.1256 - accuracy: 0.23 - ETA: 0s - loss: 2.1252 - accuracy: 0.23 - ETA: 0s - loss: 2.1246 - accuracy: 0.23 - ETA: 0s - loss: 2.1243 - accuracy: 0.23 - ETA: 0s - loss: 2.1237 - accuracy: 0.23 - ETA: 0s - loss: 2.1235 - accuracy: 0.23 - ETA: 0s - loss: 2.1234 - accuracy: 0.23 - ETA: 0s - loss: 2.1233 - accuracy: 0.23 - ETA: 0s - loss: 2.1228 - accura

Epoch 1/20
1875/1875 [==============================] - ETA: 0s - loss: 2.2521 - accuracy: 0.15 - ETA: 1s - loss: 2.2865 - accuracy: 0.09 - ETA: 1s - loss: 2.2831 - accuracy: 0.10 - ETA: 1s - loss: 2.2798 - accuracy: 0.10 - ETA: 1s - loss: 2.2759 - accuracy: 0.11 - ETA: 1s - loss: 2.2704 - accuracy: 0.12 - ETA: 1s - loss: 2.2650 - accuracy: 0.12 - ETA: 1s - loss: 2.2601 - accuracy: 0.12 - ETA: 0s - loss: 2.2552 - accuracy: 0.13 - ETA: 0s - loss: 2.2501 - accuracy: 0.13 - ETA: 0s - loss: 2.2444 - accuracy: 0.14 - ETA: 0s - loss: 2.2396 - accuracy: 0.14 - ETA: 0s - loss: 2.2352 - accuracy: 0.14 - ETA: 0s - loss: 2.2303 - accuracy: 0.15 - ETA: 0s - loss: 2.2255 - accuracy: 0.15 - ETA: 0s - loss: 2.2202 - accuracy: 0.15 - ETA: 0s - loss: 2.2157 - accuracy: 0.16 - ETA: 0s - loss: 2.2112 - accuracy: 0.16 - ETA: 0s - loss: 2.2069 - accuracy: 0.16 - ETA: 0s - loss: 2.2018 - accuracy: 0.16 - ETA: 0s - loss: 2.1975 - accuracy: 0.17 - ETA: 0s - loss: 2.1935 - accuracy: 0.17 - ETA: 0s - loss: 2.18

1875/1875 [==============================] - ETA: 0s - loss: 1.2489 - accuracy: 0.71 - ETA: 1s - loss: 0.8478 - accuracy: 0.75 - ETA: 1s - loss: 0.8673 - accuracy: 0.74 - ETA: 1s - loss: 0.8749 - accuracy: 0.73 - ETA: 1s - loss: 0.8672 - accuracy: 0.74 - ETA: 1s - loss: 0.8732 - accuracy: 0.74 - ETA: 1s - loss: 0.8778 - accuracy: 0.74 - ETA: 0s - loss: 0.8773 - accuracy: 0.74 - ETA: 0s - loss: 0.8768 - accuracy: 0.74 - ETA: 0s - loss: 0.8809 - accuracy: 0.74 - ETA: 0s - loss: 0.8812 - accuracy: 0.74 - ETA: 0s - loss: 0.8800 - accuracy: 0.74 - ETA: 0s - loss: 0.8786 - accuracy: 0.74 - ETA: 0s - loss: 0.8781 - accuracy: 0.74 - ETA: 0s - loss: 0.8759 - accuracy: 0.74 - ETA: 0s - loss: 0.8770 - accuracy: 0.74 - ETA: 0s - loss: 0.8795 - accuracy: 0.74 - ETA: 0s - loss: 0.8798 - accuracy: 0.74 - ETA: 0s - loss: 0.8791 - accuracy: 0.74 - ETA: 0s - loss: 0.8776 - accuracy: 0.74 - ETA: 0s - loss: 0.8763 - accuracy: 0.74 - ETA: 0s - loss: 0.8757 - accuracy: 0.74 - ETA: 0s - loss: 0.8762 - accura

1875/1875 [==============================] - ETA: 0s - loss: 1.1542 - accuracy: 0.59 - ETA: 1s - loss: 1.2326 - accuracy: 0.62 - ETA: 1s - loss: 1.2226 - accuracy: 0.62 - ETA: 1s - loss: 1.2104 - accuracy: 0.63 - ETA: 1s - loss: 1.2011 - accuracy: 0.63 - ETA: 1s - loss: 1.2026 - accuracy: 0.64 - ETA: 1s - loss: 1.1953 - accuracy: 0.64 - ETA: 1s - loss: 1.1932 - accuracy: 0.64 - ETA: 0s - loss: 1.1916 - accuracy: 0.64 - ETA: 0s - loss: 1.1915 - accuracy: 0.64 - ETA: 0s - loss: 1.1908 - accuracy: 0.64 - ETA: 0s - loss: 1.1886 - accuracy: 0.64 - ETA: 0s - loss: 1.1886 - accuracy: 0.64 - ETA: 0s - loss: 1.1866 - accuracy: 0.64 - ETA: 0s - loss: 1.1870 - accuracy: 0.64 - ETA: 0s - loss: 1.1835 - accuracy: 0.64 - ETA: 0s - loss: 1.1819 - accuracy: 0.64 - ETA: 0s - loss: 1.1768 - accuracy: 0.64 - ETA: 0s - loss: 1.1762 - accuracy: 0.64 - ETA: 0s - loss: 1.1740 - accuracy: 0.65 - ETA: 0s - loss: 1.1712 - accuracy: 0.65 - ETA: 0s - loss: 1.1707 - accuracy: 0.65 - ETA: 0s - loss: 1.1678 - accura

1875/1875 [==============================] - ETA: 0s - loss: 0.7499 - accuracy: 0.71 - ETA: 1s - loss: 0.8803 - accuracy: 0.74 - ETA: 1s - loss: 0.8537 - accuracy: 0.75 - ETA: 1s - loss: 0.8448 - accuracy: 0.75 - ETA: 1s - loss: 0.8337 - accuracy: 0.75 - ETA: 1s - loss: 0.8363 - accuracy: 0.75 - ETA: 1s - loss: 0.8426 - accuracy: 0.75 - ETA: 1s - loss: 0.8442 - accuracy: 0.75 - ETA: 1s - loss: 0.8407 - accuracy: 0.75 - ETA: 1s - loss: 0.8394 - accuracy: 0.75 - ETA: 0s - loss: 0.8369 - accuracy: 0.75 - ETA: 0s - loss: 0.8373 - accuracy: 0.75 - ETA: 0s - loss: 0.8376 - accuracy: 0.75 - ETA: 0s - loss: 0.8342 - accuracy: 0.75 - ETA: 0s - loss: 0.8354 - accuracy: 0.75 - ETA: 0s - loss: 0.8359 - accuracy: 0.75 - ETA: 0s - loss: 0.8355 - accuracy: 0.75 - ETA: 0s - loss: 0.8343 - accuracy: 0.75 - ETA: 0s - loss: 0.8334 - accuracy: 0.75 - ETA: 0s - loss: 0.8334 - accuracy: 0.75 - ETA: 0s - loss: 0.8326 - accuracy: 0.75 - ETA: 0s - loss: 0.8327 - accuracy: 0.75 - ETA: 0s - loss: 0.8319 - accura

{'learning_rate': 0.001, 'momentum': 0.4}


In [46]:
def tune_neurons_model(hp):
    model = keras.Sequential()
    model.add(keras.layers.Dense(units=hp.Int('units',
                                        min_value=8,
                                        max_value=128,
                                        step=16),
                                 activation="relu", 
                                 input_shape=[X_train.shape[1]]))

    model.add(keras.layers.Dense(1, activation='sigmoid'))

    model.compile(
        optimizer="adam",
        loss = 'sparse_categorical_crossentropy', 
        metrics = ['accuracy'])
    return model

In [47]:
random_search_params(tune_neurons_model, project_name="tune_neurons")

Epoch 1/20


InvalidArgumentError:  Received a label value of 9 which is outside the valid range of [0, 1).  Label values: 3 0 9 7 7 7 1 9 7 5 1 1 7 4 7 0 4 4 5 0 6 0 0 4 0 7 2 2 5 5 7 2
	 [[node sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits (defined at /opt/anaconda3/envs/U4-S1-NLP/lib/python3.8/site-packages/wandb/integration/keras/keras.py:120) ]] [Op:__inference_train_function_3271959]

Function call stack:
train_function


In [48]:
def tune_layers_model(hp):
    model = keras.Sequential()

    model.add(keras.layers.Dense(units=128,
                                 activation="relu", 
                                 input_shape=[X_train.shape[1]]))

    for i in range(hp.Int('num_layers', 1, 6)):
      model.add(keras.layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=8,
                                            max_value=64,
                                            step=8),
                               activation='relu'))

    model.add(keras.layers.Dense(1, activation='sigmoid'))

    model.compile(
        optimizer="adam",
        loss = 'sparse_categorical_crossentropy', 
        metrics = ['accuracy'])
    return model

In [49]:
random_search_params(tune_layers_model, project_name="tune_layers")

Epoch 1/20


InvalidArgumentError:  Received a label value of 9 which is outside the valid range of [0, 1).  Label values: 9 0 7 1 1 2 6 2 9 3 8 8 2 9 5 1 6 7 4 2 5 4 1 0 1 2 2 6 2 7 8 4
	 [[node sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits (defined at /opt/anaconda3/envs/U4-S1-NLP/lib/python3.8/site-packages/wandb/integration/keras/keras.py:120) ]] [Op:__inference_train_function_3273016]

Function call stack:
train_function


In [50]:
def tune_weight_init_model(hp):
    model = keras.Sequential()

    weight_init = hp.Choice('weight_init', 
                            ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform'])

    model.add(keras.layers.Dense(units=32,
                                 activation="relu",
                                 kernel_initializer=weight_init,
                                 input_shape=[X_train.shape[1]]))

    model.add(keras.layers.Dense(1, activation='sigmoid'))

    model.compile(
        optimizer="adam",
        loss = 'sparse_categorical_crossentropy', 
        metrics = ['accuracy'])
    return model

In [51]:
random_search_params(tune_weight_init_model, project_name="tune_weight_init")

Epoch 1/20


InvalidArgumentError:  Received a label value of 9 which is outside the valid range of [0, 1).  Label values: 0 1 5 8 6 3 4 7 7 0 7 3 9 1 1 5 6 5 2 3 1 2 8 4 5 6 1 5 3 9 7 1
	 [[node sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits (defined at /opt/anaconda3/envs/U4-S1-NLP/lib/python3.8/site-packages/wandb/integration/keras/keras.py:120) ]] [Op:__inference_train_function_3273700]

Function call stack:
train_function


# Activation function

In [52]:
def tune_act_model(hp):
    model = keras.Sequential()

    activation = hp.Choice('activation', 
                            ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear'])

    model.add(keras.layers.Dense(units=32,
                                 activation=activation,
                                 input_shape=[X_train.shape[1]]))

    model.add(keras.layers.Dense(1, activation='sigmoid'))

    model.compile(
        optimizer="adam",
        loss = 'sparse_categorical_crossentropy', 
        metrics = ['accuracy'])
    return model

In [53]:
random_search_params(tune_act_model, "tune_activation")

Epoch 1/20


InvalidArgumentError:  Received a label value of 9 which is outside the valid range of [0, 1).  Label values: 9 6 7 0 5 6 4 6 0 1 3 8 2 6 3 3 5 6 7 0 0 7 2 0 8 7 4 6 1 3 7 6
	 [[node sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits (defined at /opt/anaconda3/envs/U4-S1-NLP/lib/python3.8/site-packages/wandb/integration/keras/keras.py:120) ]] [Op:__inference_train_function_3274386]

Function call stack:
train_function


# Dropout rate

In [54]:
def tune_dropout_model(hp):
    model = keras.Sequential()

    drop_rate = hp.Choice('drop_rate', 
                            [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])

    model.add(keras.layers.Dense(units=32,
                                 activation="relu",
                                 input_shape=[X_train.shape[1]]))
    model.add(keras.layers.Dropout(rate=drop_rate))

    model.add(keras.layers.Dense(1, activation='sigmoid'))

    model.compile(
        optimizer="adam",
        loss = 'sparse_categorical_crossentropy', 
        metrics = ['accuracy'])
    return model

In [55]:
random_search_params(tune_dropout_model, "tune_dropout")

Epoch 1/20


InvalidArgumentError:  Received a label value of 9 which is outside the valid range of [0, 1).  Label values: 3 7 0 2 4 9 7 3 6 6 4 2 0 2 9 9 0 4 2 5 7 9 3 9 3 1 2 0 3 9 4 2
	 [[node sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits (defined at /opt/anaconda3/envs/U4-S1-NLP/lib/python3.8/site-packages/wandb/integration/keras/keras.py:120) ]] [Op:__inference_train_function_3275126]

Function call stack:
train_function


In [60]:
def tune_nn_model(hp):
    model = keras.Sequential()

    model.add(keras.layers.Dense(units=128,
                                 activation="relu", 
                                 input_shape=[X_train.shape[1]]))

    for i in range(hp.Int('num_layers', 1, 6)):
      units = hp.Int(
          'units_' + str(i), 
          min_value=8,
          max_value=64,
          step=8
      )
      model.add(keras.layers.Dense(units=units, activation='relu'))
      drop_rate = hp.Choice('drop_rate_' + str(i), 
                            [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])
      model.add(keras.layers.Dropout(rate=drop_rate))

    model.add(keras.layers.Dense(10, activation='sigmoid'))

    model.compile(
        optimizer="adam",
        loss = 'sparse_categorical_crossentropy', 
        metrics = ['accuracy'])
    return model

In [61]:
random_search_params(tune_nn_model, "tune_nn")

Epoch 1/20
1875/1875 [==============================] - ETA: 0s - loss: 2.3034 - accuracy: 0.09 - ETA: 2s - loss: 2.3020 - accuracy: 0.09 - ETA: 2s - loss: 2.2943 - accuracy: 0.10 - ETA: 1s - loss: 2.2735 - accuracy: 0.11 - ETA: 1s - loss: 2.2496 - accuracy: 0.12 - ETA: 1s - loss: 2.2278 - accuracy: 0.12 - ETA: 1s - loss: 2.2086 - accuracy: 0.13 - ETA: 1s - loss: 2.1941 - accuracy: 0.13 - ETA: 1s - loss: 2.1797 - accuracy: 0.14 - ETA: 1s - loss: 2.1628 - accuracy: 0.14 - ETA: 1s - loss: 2.1528 - accuracy: 0.15 - ETA: 1s - loss: 2.1410 - accuracy: 0.15 - ETA: 1s - loss: 2.1274 - accuracy: 0.16 - ETA: 1s - loss: 2.1159 - accuracy: 0.16 - ETA: 1s - loss: 2.1060 - accuracy: 0.16 - ETA: 1s - loss: 2.0969 - accuracy: 0.17 - ETA: 1s - loss: 2.0876 - accuracy: 0.17 - ETA: 1s - loss: 2.0765 - accuracy: 0.17 - ETA: 1s - loss: 2.0692 - accuracy: 0.18 - ETA: 1s - loss: 2.0603 - accuracy: 0.18 - ETA: 1s - loss: 2.0504 - accuracy: 0.18 - ETA: 0s - loss: 2.0432 - accuracy: 0.19 - ETA: 0s - loss: 2.03

1875/1875 [==============================] - ETA: 0s - loss: 1.2944 - accuracy: 0.56 - ETA: 1s - loss: 1.2045 - accuracy: 0.59 - ETA: 1s - loss: 1.1629 - accuracy: 0.61 - ETA: 1s - loss: 1.1571 - accuracy: 0.62 - ETA: 1s - loss: 1.1543 - accuracy: 0.62 - ETA: 1s - loss: 1.1491 - accuracy: 0.62 - ETA: 1s - loss: 1.1555 - accuracy: 0.62 - ETA: 1s - loss: 1.1455 - accuracy: 0.62 - ETA: 1s - loss: 1.1462 - accuracy: 0.62 - ETA: 1s - loss: 1.1523 - accuracy: 0.62 - ETA: 1s - loss: 1.1541 - accuracy: 0.62 - ETA: 1s - loss: 1.1490 - accuracy: 0.62 - ETA: 1s - loss: 1.1551 - accuracy: 0.62 - ETA: 1s - loss: 1.1556 - accuracy: 0.62 - ETA: 1s - loss: 1.1557 - accuracy: 0.62 - ETA: 1s - loss: 1.1567 - accuracy: 0.62 - ETA: 1s - loss: 1.1596 - accuracy: 0.62 - ETA: 1s - loss: 1.1616 - accuracy: 0.62 - ETA: 1s - loss: 1.1602 - accuracy: 0.62 - ETA: 1s - loss: 1.1609 - accuracy: 0.62 - ETA: 0s - loss: 1.1618 - accuracy: 0.62 - ETA: 0s - loss: 1.1628 - accuracy: 0.62 - ETA: 0s - loss: 1.1631 - accura

1875/1875 [==============================] - ETA: 0s - loss: 1.2569 - accuracy: 0.53 - ETA: 2s - loss: 1.0241 - accuracy: 0.66 - ETA: 1s - loss: 1.0416 - accuracy: 0.66 - ETA: 1s - loss: 1.0357 - accuracy: 0.66 - ETA: 1s - loss: 1.0320 - accuracy: 0.67 - ETA: 1s - loss: 1.0305 - accuracy: 0.66 - ETA: 1s - loss: 1.0322 - accuracy: 0.66 - ETA: 1s - loss: 1.0415 - accuracy: 0.66 - ETA: 1s - loss: 1.0355 - accuracy: 0.66 - ETA: 1s - loss: 1.0314 - accuracy: 0.66 - ETA: 1s - loss: 1.0308 - accuracy: 0.66 - ETA: 1s - loss: 1.0326 - accuracy: 0.66 - ETA: 1s - loss: 1.0306 - accuracy: 0.66 - ETA: 1s - loss: 1.0333 - accuracy: 0.66 - ETA: 1s - loss: 1.0323 - accuracy: 0.66 - ETA: 1s - loss: 1.0338 - accuracy: 0.66 - ETA: 1s - loss: 1.0317 - accuracy: 0.66 - ETA: 1s - loss: 1.0322 - accuracy: 0.66 - ETA: 1s - loss: 1.0361 - accuracy: 0.66 - ETA: 1s - loss: 1.0357 - accuracy: 0.66 - ETA: 0s - loss: 1.0369 - accuracy: 0.66 - ETA: 0s - loss: 1.0400 - accuracy: 0.66 - ETA: 0s - loss: 1.0403 - accura

1875/1875 [==============================] - ETA: 0s - loss: 1.5731 - accuracy: 0.37 - ETA: 2s - loss: 1.3801 - accuracy: 0.51 - ETA: 2s - loss: 1.3537 - accuracy: 0.51 - ETA: 2s - loss: 1.3686 - accuracy: 0.50 - ETA: 2s - loss: 1.3695 - accuracy: 0.51 - ETA: 1s - loss: 1.3740 - accuracy: 0.51 - ETA: 1s - loss: 1.3729 - accuracy: 0.50 - ETA: 1s - loss: 1.3725 - accuracy: 0.50 - ETA: 1s - loss: 1.3682 - accuracy: 0.50 - ETA: 1s - loss: 1.3651 - accuracy: 0.50 - ETA: 1s - loss: 1.3727 - accuracy: 0.50 - ETA: 1s - loss: 1.3699 - accuracy: 0.50 - ETA: 1s - loss: 1.3693 - accuracy: 0.50 - ETA: 1s - loss: 1.3698 - accuracy: 0.50 - ETA: 1s - loss: 1.3669 - accuracy: 0.50 - ETA: 1s - loss: 1.3639 - accuracy: 0.50 - ETA: 1s - loss: 1.3672 - accuracy: 0.50 - ETA: 1s - loss: 1.3694 - accuracy: 0.50 - ETA: 1s - loss: 1.3707 - accuracy: 0.50 - ETA: 1s - loss: 1.3706 - accuracy: 0.50 - ETA: 1s - loss: 1.3719 - accuracy: 0.50 - ETA: 1s - loss: 1.3708 - accuracy: 0.50 - ETA: 1s - loss: 1.3692 - accura

1875/1875 [==============================] - ETA: 0s - loss: 1.3609 - accuracy: 0.53 - ETA: 2s - loss: 1.1472 - accuracy: 0.61 - ETA: 2s - loss: 1.1764 - accuracy: 0.59 - ETA: 2s - loss: 1.1783 - accuracy: 0.60 - ETA: 2s - loss: 1.1771 - accuracy: 0.60 - ETA: 2s - loss: 1.1694 - accuracy: 0.60 - ETA: 2s - loss: 1.1545 - accuracy: 0.60 - ETA: 2s - loss: 1.1568 - accuracy: 0.60 - ETA: 1s - loss: 1.1601 - accuracy: 0.60 - ETA: 1s - loss: 1.1580 - accuracy: 0.60 - ETA: 1s - loss: 1.1590 - accuracy: 0.60 - ETA: 1s - loss: 1.1550 - accuracy: 0.60 - ETA: 1s - loss: 1.1543 - accuracy: 0.60 - ETA: 1s - loss: 1.1528 - accuracy: 0.60 - ETA: 1s - loss: 1.1501 - accuracy: 0.60 - ETA: 1s - loss: 1.1569 - accuracy: 0.60 - ETA: 1s - loss: 1.1565 - accuracy: 0.60 - ETA: 1s - loss: 1.1586 - accuracy: 0.60 - ETA: 1s - loss: 1.1590 - accuracy: 0.60 - ETA: 1s - loss: 1.1604 - accuracy: 0.60 - ETA: 1s - loss: 1.1589 - accuracy: 0.60 - ETA: 1s - loss: 1.1596 - accuracy: 0.60 - ETA: 1s - loss: 1.1589 - accura

...

1875/1875 [==============================] - ETA: 0s - loss: 0.2806 - accuracy: 0.87 - ETA: 2s - loss: 0.2644 - accuracy: 0.91 - ETA: 2s - loss: 0.2665 - accuracy: 0.91 - ETA: 1s - loss: 0.2718 - accuracy: 0.91 - ETA: 1s - loss: 0.2864 - accuracy: 0.91 - ETA: 1s - loss: 0.2953 - accuracy: 0.91 - ETA: 1s - loss: 0.2892 - accuracy: 0.91 - ETA: 1s - loss: 0.2900 - accuracy: 0.91 - ETA: 1s - loss: 0.2896 - accuracy: 0.91 - ETA: 1s - loss: 0.2886 - accuracy: 0.91 - ETA: 1s - loss: 0.2869 - accuracy: 0.91 - ETA: 1s - loss: 0.2927 - accuracy: 0.91 - ETA: 1s - loss: 0.2916 - accuracy: 0.91 - ETA: 1s - loss: 0.2915 - accuracy: 0.91 - ETA: 1s - loss: 0.2933 - accuracy: 0.91 - ETA: 1s - loss: 0.2955 - accuracy: 0.91 - ETA: 1s - loss: 0.2956 - accuracy: 0.91 - ETA: 1s - loss: 0.2952 - accuracy: 0.91 - ETA: 1s - loss: 0.2960 - accuracy: 0.91 - ETA: 1s - loss: 0.2987 - accuracy: 0.91 - ETA: 1s - loss: 0.2971 - accuracy: 0.91 - ETA: 1s - loss: 0.2979 - accuracy: 0.91 - ETA: 1s - loss: 0.2991 - accura

1875/1875 [==============================] - ETA: 0s - loss: 0.6591 - accuracy: 0.87 - ETA: 3s - loss: 0.6318 - accuracy: 0.82 - ETA: 3s - loss: 0.5966 - accuracy: 0.82 - ETA: 2s - loss: 0.5892 - accuracy: 0.82 - ETA: 2s - loss: 0.5924 - accuracy: 0.82 - ETA: 2s - loss: 0.6050 - accuracy: 0.82 - ETA: 2s - loss: 0.6052 - accuracy: 0.82 - ETA: 2s - loss: 0.6029 - accuracy: 0.82 - ETA: 2s - loss: 0.5999 - accuracy: 0.82 - ETA: 2s - loss: 0.5980 - accuracy: 0.82 - ETA: 2s - loss: 0.5977 - accuracy: 0.82 - ETA: 2s - loss: 0.5921 - accuracy: 0.82 - ETA: 2s - loss: 0.5958 - accuracy: 0.82 - ETA: 2s - loss: 0.5953 - accuracy: 0.82 - ETA: 2s - loss: 0.5933 - accuracy: 0.82 - ETA: 2s - loss: 0.5948 - accuracy: 0.82 - ETA: 2s - loss: 0.5948 - accuracy: 0.82 - ETA: 2s - loss: 0.5937 - accuracy: 0.82 - ETA: 2s - loss: 0.5895 - accuracy: 0.82 - ETA: 2s - loss: 0.5862 - accuracy: 0.82 - ETA: 2s - loss: 0.5866 - accuracy: 0.82 - ETA: 2s - loss: 0.5864 - accuracy: 0.82 - ETA: 2s - loss: 0.5876 - accura

1875/1875 [==============================] - ETA: 0s - loss: 0.6319 - accuracy: 0.90 - ETA: 1s - loss: 0.4029 - accuracy: 0.88 - ETA: 2s - loss: 0.4000 - accuracy: 0.88 - ETA: 2s - loss: 0.3943 - accuracy: 0.88 - ETA: 2s - loss: 0.3841 - accuracy: 0.88 - ETA: 2s - loss: 0.3756 - accuracy: 0.88 - ETA: 2s - loss: 0.3786 - accuracy: 0.88 - ETA: 2s - loss: 0.3832 - accuracy: 0.88 - ETA: 2s - loss: 0.3849 - accuracy: 0.88 - ETA: 2s - loss: 0.3839 - accuracy: 0.88 - ETA: 2s - loss: 0.3861 - accuracy: 0.88 - ETA: 2s - loss: 0.3868 - accuracy: 0.88 - ETA: 2s - loss: 0.3871 - accuracy: 0.88 - ETA: 2s - loss: 0.3913 - accuracy: 0.88 - ETA: 2s - loss: 0.3929 - accuracy: 0.88 - ETA: 1s - loss: 0.3932 - accuracy: 0.88 - ETA: 1s - loss: 0.3935 - accuracy: 0.88 - ETA: 1s - loss: 0.3908 - accuracy: 0.88 - ETA: 1s - loss: 0.3913 - accuracy: 0.88 - ETA: 1s - loss: 0.3902 - accuracy: 0.88 - ETA: 1s - loss: 0.3874 - accuracy: 0.88 - ETA: 1s - loss: 0.3894 - accuracy: 0.88 - ETA: 1s - loss: 0.3898 - accura

1875/1875 [==============================] - ETA: 0s - loss: 0.1420 - accuracy: 0.96 - ETA: 1s - loss: 0.3270 - accuracy: 0.89 - ETA: 1s - loss: 0.2856 - accuracy: 0.90 - ETA: 1s - loss: 0.2845 - accuracy: 0.90 - ETA: 1s - loss: 0.2839 - accuracy: 0.91 - ETA: 1s - loss: 0.2808 - accuracy: 0.91 - ETA: 1s - loss: 0.2809 - accuracy: 0.91 - ETA: 1s - loss: 0.2832 - accuracy: 0.91 - ETA: 1s - loss: 0.2918 - accuracy: 0.91 - ETA: 1s - loss: 0.2950 - accuracy: 0.91 - ETA: 1s - loss: 0.2961 - accuracy: 0.91 - ETA: 1s - loss: 0.2993 - accuracy: 0.90 - ETA: 1s - loss: 0.2990 - accuracy: 0.90 - ETA: 1s - loss: 0.3009 - accuracy: 0.90 - ETA: 1s - loss: 0.3032 - accuracy: 0.90 - ETA: 1s - loss: 0.3029 - accuracy: 0.90 - ETA: 1s - loss: 0.3028 - accuracy: 0.90 - ETA: 1s - loss: 0.3016 - accuracy: 0.90 - ETA: 1s - loss: 0.3008 - accuracy: 0.91 - ETA: 1s - loss: 0.3011 - accuracy: 0.91 - ETA: 1s - loss: 0.3013 - accuracy: 0.91 - ETA: 1s - loss: 0.3023 - accuracy: 0.91 - ETA: 1s - loss: 0.3032 - accura

{'drop_rate_0': 0.5,
 'drop_rate_1': 0.1,
 'drop_rate_2': 0.3,
 'drop_rate_3': 0.9,
 'num_layers': 2,
 'units_0': 56,
 'units_1': 40,
 'units_2': 32,
 'units_3': 64}


In [62]:
b_tuner = BayesianOptimization(
    tune_nn_model,
    objective='val_accuracy',
    max_trials=MAX_TRIALS,
    executions_per_trial=EXECUTIONS_PER_TRIAL,
    directory='test_dir', 
    project_name='b_tune_nn',
    seed=RANDOM_SEED
)

b_tuner.search_space_summary()

In [63]:
b_tuner.search(x=X_train,
             y=y_train,
             epochs=TRAIN_EPOCHS,
             validation_data=(X_test, y_test))
  
b_tuner.results_summary()

pprint(b_tuner.oracle.get_best_trials(num_trials=1)[0].hyperparameters.values)

Epoch 1/20
1875/1875 [==============================] - ETA: 0s - loss: 2.2702 - accuracy: 0.09 - ETA: 2s - loss: 2.2698 - accuracy: 0.12 - ETA: 2s - loss: 2.2497 - accuracy: 0.12 - ETA: 2s - loss: 2.2435 - accuracy: 0.14 - ETA: 2s - loss: 2.2244 - accuracy: 0.15 - ETA: 1s - loss: 2.2108 - accuracy: 0.15 - ETA: 1s - loss: 2.1972 - accuracy: 0.15 - ETA: 1s - loss: 2.1901 - accuracy: 0.15 - ETA: 1s - loss: 2.1791 - accuracy: 0.16 - ETA: 1s - loss: 2.1704 - accuracy: 0.16 - ETA: 1s - loss: 2.1590 - accuracy: 0.16 - ETA: 1s - loss: 2.1459 - accuracy: 0.16 - ETA: 1s - loss: 2.1389 - accuracy: 0.16 - ETA: 1s - loss: 2.1317 - accuracy: 0.16 - ETA: 1s - loss: 2.1226 - accuracy: 0.17 - ETA: 1s - loss: 2.1156 - accuracy: 0.17 - ETA: 1s - loss: 2.1107 - accuracy: 0.17 - ETA: 1s - loss: 2.1058 - accuracy: 0.17 - ETA: 1s - loss: 2.0996 - accuracy: 0.17 - ETA: 1s - loss: 2.0931 - accuracy: 0.17 - ETA: 1s - loss: 2.0840 - accuracy: 0.17 - ETA: 1s - loss: 2.0751 - accuracy: 0.18 - ETA: 1s - loss: 2.06

1875/1875 [==============================] - ETA: 0s - loss: 1.4331 - accuracy: 0.50 - ETA: 2s - loss: 1.1834 - accuracy: 0.62 - ETA: 2s - loss: 1.1664 - accuracy: 0.62 - ETA: 2s - loss: 1.1671 - accuracy: 0.62 - ETA: 2s - loss: 1.1660 - accuracy: 0.63 - ETA: 2s - loss: 1.1633 - accuracy: 0.63 - ETA: 1s - loss: 1.1736 - accuracy: 0.62 - ETA: 1s - loss: 1.1756 - accuracy: 0.62 - ETA: 1s - loss: 1.1734 - accuracy: 0.62 - ETA: 1s - loss: 1.1732 - accuracy: 0.62 - ETA: 1s - loss: 1.1698 - accuracy: 0.63 - ETA: 1s - loss: 1.1674 - accuracy: 0.63 - ETA: 1s - loss: 1.1687 - accuracy: 0.63 - ETA: 1s - loss: 1.1710 - accuracy: 0.62 - ETA: 1s - loss: 1.1690 - accuracy: 0.63 - ETA: 1s - loss: 1.1731 - accuracy: 0.63 - ETA: 1s - loss: 1.1742 - accuracy: 0.63 - ETA: 1s - loss: 1.1736 - accuracy: 0.63 - ETA: 1s - loss: 1.1755 - accuracy: 0.62 - ETA: 1s - loss: 1.1761 - accuracy: 0.62 - ETA: 1s - loss: 1.1763 - accuracy: 0.62 - ETA: 1s - loss: 1.1740 - accuracy: 0.62 - ETA: 1s - loss: 1.1762 - accura

Epoch 13/20
1875/1875 [==============================] - ETA: 0s - loss: 0.8010 - accuracy: 0.68 - ETA: 2s - loss: 1.1596 - accuracy: 0.64 - ETA: 2s - loss: 1.1560 - accuracy: 0.63 - ETA: 2s - loss: 1.1241 - accuracy: 0.64 - ETA: 1s - loss: 1.1102 - accuracy: 0.64 - ETA: 1s - loss: 1.1099 - accuracy: 0.65 - ETA: 1s - loss: 1.1071 - accuracy: 0.65 - ETA: 1s - loss: 1.1152 - accuracy: 0.64 - ETA: 1s - loss: 1.1127 - accuracy: 0.65 - ETA: 1s - loss: 1.1108 - accuracy: 0.65 - ETA: 1s - loss: 1.1109 - accuracy: 0.65 - ETA: 1s - loss: 1.1070 - accuracy: 0.65 - ETA: 1s - loss: 1.1093 - accuracy: 0.65 - ETA: 1s - loss: 1.1085 - accuracy: 0.65 - ETA: 1s - loss: 1.1076 - accuracy: 0.65 - ETA: 1s - loss: 1.1058 - accuracy: 0.65 - ETA: 1s - loss: 1.1044 - accuracy: 0.65 - ETA: 1s - loss: 1.1050 - accuracy: 0.65 - ETA: 1s - loss: 1.1041 - accuracy: 0.65 - ETA: 1s - loss: 1.1044 - accuracy: 0.65 - ETA: 1s - loss: 1.1008 - accuracy: 0.65 - ETA: 1s - loss: 1.1019 - accuracy: 0.65 - ETA: 1s - loss: 1.1

1875/1875 [==============================] - ETA: 0s - loss: 2.3017 - accuracy: 0.06 - ETA: 2s - loss: 2.2660 - accuracy: 0.12 - ETA: 2s - loss: 2.2421 - accuracy: 0.13 - ETA: 2s - loss: 2.2146 - accuracy: 0.14 - ETA: 2s - loss: 2.2025 - accuracy: 0.14 - ETA: 2s - loss: 2.1747 - accuracy: 0.16 - ETA: 2s - loss: 2.1559 - accuracy: 0.16 - ETA: 2s - loss: 2.1363 - accuracy: 0.17 - ETA: 2s - loss: 2.1188 - accuracy: 0.18 - ETA: 2s - loss: 2.1015 - accuracy: 0.18 - ETA: 1s - loss: 2.0894 - accuracy: 0.19 - ETA: 1s - loss: 2.0739 - accuracy: 0.19 - ETA: 1s - loss: 2.0573 - accuracy: 0.20 - ETA: 1s - loss: 2.0485 - accuracy: 0.20 - ETA: 1s - loss: 2.0392 - accuracy: 0.20 - ETA: 1s - loss: 2.0300 - accuracy: 0.20 - ETA: 1s - loss: 2.0210 - accuracy: 0.21 - ETA: 1s - loss: 2.0131 - accuracy: 0.21 - ETA: 1s - loss: 2.0086 - accuracy: 0.21 - ETA: 1s - loss: 2.0032 - accuracy: 0.21 - ETA: 1s - loss: 1.9956 - accuracy: 0.21 - ETA: 1s - loss: 1.9905 - accuracy: 0.21 - ETA: 1s - loss: 1.9845 - accura

1875/1875 [==============================] - ETA: 0s - loss: 1.4287 - accuracy: 0.50 - ETA: 1s - loss: 1.3543 - accuracy: 0.44 - ETA: 1s - loss: 1.3497 - accuracy: 0.46 - ETA: 1s - loss: 1.3541 - accuracy: 0.46 - ETA: 1s - loss: 1.3530 - accuracy: 0.45 - ETA: 1s - loss: 1.3632 - accuracy: 0.45 - ETA: 1s - loss: 1.3592 - accuracy: 0.45 - ETA: 1s - loss: 1.3587 - accuracy: 0.45 - ETA: 1s - loss: 1.3603 - accuracy: 0.45 - ETA: 1s - loss: 1.3612 - accuracy: 0.45 - ETA: 1s - loss: 1.3559 - accuracy: 0.45 - ETA: 1s - loss: 1.3539 - accuracy: 0.45 - ETA: 1s - loss: 1.3545 - accuracy: 0.45 - ETA: 1s - loss: 1.3553 - accuracy: 0.45 - ETA: 1s - loss: 1.3530 - accuracy: 0.45 - ETA: 1s - loss: 1.3522 - accuracy: 0.45 - ETA: 1s - loss: 1.3522 - accuracy: 0.45 - ETA: 1s - loss: 1.3520 - accuracy: 0.45 - ETA: 1s - loss: 1.3525 - accuracy: 0.45 - ETA: 1s - loss: 1.3516 - accuracy: 0.45 - ETA: 1s - loss: 1.3499 - accuracy: 0.45 - ETA: 0s - loss: 1.3517 - accuracy: 0.45 - ETA: 0s - loss: 1.3523 - accura

...

1875/1875 [==============================] - ETA: 0s - loss: 0.4526 - accuracy: 0.81 - ETA: 1s - loss: 0.3107 - accuracy: 0.90 - ETA: 1s - loss: 0.2987 - accuracy: 0.91 - ETA: 1s - loss: 0.2956 - accuracy: 0.91 - ETA: 1s - loss: 0.2940 - accuracy: 0.91 - ETA: 1s - loss: 0.2937 - accuracy: 0.91 - ETA: 1s - loss: 0.2909 - accuracy: 0.91 - ETA: 1s - loss: 0.2899 - accuracy: 0.91 - ETA: 1s - loss: 0.2931 - accuracy: 0.91 - ETA: 1s - loss: 0.2909 - accuracy: 0.91 - ETA: 1s - loss: 0.2916 - accuracy: 0.91 - ETA: 1s - loss: 0.2954 - accuracy: 0.91 - ETA: 1s - loss: 0.2948 - accuracy: 0.91 - ETA: 1s - loss: 0.2931 - accuracy: 0.91 - ETA: 1s - loss: 0.2928 - accuracy: 0.91 - ETA: 1s - loss: 0.2937 - accuracy: 0.91 - ETA: 1s - loss: 0.2951 - accuracy: 0.91 - ETA: 1s - loss: 0.2949 - accuracy: 0.91 - ETA: 1s - loss: 0.2945 - accuracy: 0.91 - ETA: 1s - loss: 0.2953 - accuracy: 0.91 - ETA: 0s - loss: 0.2954 - accuracy: 0.91 - ETA: 0s - loss: 0.2977 - accuracy: 0.91 - ETA: 0s - loss: 0.2960 - accura

1875/1875 [==============================] - ETA: 0s - loss: 0.5012 - accuracy: 0.87 - ETA: 2s - loss: 0.4464 - accuracy: 0.86 - ETA: 2s - loss: 0.4657 - accuracy: 0.85 - ETA: 1s - loss: 0.4560 - accuracy: 0.86 - ETA: 1s - loss: 0.4511 - accuracy: 0.85 - ETA: 1s - loss: 0.4577 - accuracy: 0.86 - ETA: 1s - loss: 0.4641 - accuracy: 0.85 - ETA: 1s - loss: 0.4706 - accuracy: 0.85 - ETA: 1s - loss: 0.4736 - accuracy: 0.85 - ETA: 1s - loss: 0.4799 - accuracy: 0.85 - ETA: 1s - loss: 0.4824 - accuracy: 0.85 - ETA: 1s - loss: 0.4799 - accuracy: 0.85 - ETA: 1s - loss: 0.4815 - accuracy: 0.85 - ETA: 1s - loss: 0.4792 - accuracy: 0.85 - ETA: 1s - loss: 0.4822 - accuracy: 0.85 - ETA: 1s - loss: 0.4836 - accuracy: 0.85 - ETA: 1s - loss: 0.4817 - accuracy: 0.85 - ETA: 1s - loss: 0.4831 - accuracy: 0.85 - ETA: 1s - loss: 0.4821 - accuracy: 0.85 - ETA: 1s - loss: 0.4839 - accuracy: 0.85 - ETA: 1s - loss: 0.4843 - accuracy: 0.85 - ETA: 1s - loss: 0.4865 - accuracy: 0.85 - ETA: 0s - loss: 0.4875 - accura

1875/1875 [==============================] - ETA: 0s - loss: 0.3773 - accuracy: 0.87 - ETA: 1s - loss: 0.3349 - accuracy: 0.90 - ETA: 1s - loss: 0.3217 - accuracy: 0.91 - ETA: 2s - loss: 0.3262 - accuracy: 0.90 - ETA: 2s - loss: 0.3356 - accuracy: 0.90 - ETA: 2s - loss: 0.3354 - accuracy: 0.90 - ETA: 2s - loss: 0.3313 - accuracy: 0.90 - ETA: 2s - loss: 0.3274 - accuracy: 0.90 - ETA: 2s - loss: 0.3277 - accuracy: 0.90 - ETA: 1s - loss: 0.3330 - accuracy: 0.90 - ETA: 1s - loss: 0.3336 - accuracy: 0.90 - ETA: 1s - loss: 0.3334 - accuracy: 0.90 - ETA: 1s - loss: 0.3365 - accuracy: 0.90 - ETA: 1s - loss: 0.3362 - accuracy: 0.90 - ETA: 1s - loss: 0.3375 - accuracy: 0.90 - ETA: 1s - loss: 0.3398 - accuracy: 0.90 - ETA: 1s - loss: 0.3361 - accuracy: 0.90 - ETA: 1s - loss: 0.3368 - accuracy: 0.90 - ETA: 1s - loss: 0.3359 - accuracy: 0.90 - ETA: 1s - loss: 0.3394 - accuracy: 0.90 - ETA: 1s - loss: 0.3413 - accuracy: 0.89 - ETA: 1s - loss: 0.3419 - accuracy: 0.89 - ETA: 1s - loss: 0.3434 - accura

1875/1875 [==============================] - ETA: 0s - loss: 0.4007 - accuracy: 0.87 - ETA: 2s - loss: 0.2726 - accuracy: 0.91 - ETA: 2s - loss: 0.2674 - accuracy: 0.91 - ETA: 2s - loss: 0.2736 - accuracy: 0.91 - ETA: 2s - loss: 0.2721 - accuracy: 0.92 - ETA: 2s - loss: 0.2743 - accuracy: 0.92 - ETA: 2s - loss: 0.2736 - accuracy: 0.92 - ETA: 2s - loss: 0.2723 - accuracy: 0.92 - ETA: 2s - loss: 0.2705 - accuracy: 0.92 - ETA: 2s - loss: 0.2758 - accuracy: 0.92 - ETA: 2s - loss: 0.2752 - accuracy: 0.92 - ETA: 2s - loss: 0.2766 - accuracy: 0.91 - ETA: 2s - loss: 0.2776 - accuracy: 0.91 - ETA: 2s - loss: 0.2776 - accuracy: 0.91 - ETA: 2s - loss: 0.2812 - accuracy: 0.91 - ETA: 2s - loss: 0.2816 - accuracy: 0.91 - ETA: 2s - loss: 0.2833 - accuracy: 0.91 - ETA: 2s - loss: 0.2856 - accuracy: 0.91 - ETA: 2s - loss: 0.2865 - accuracy: 0.91 - ETA: 2s - loss: 0.2885 - accuracy: 0.91 - ETA: 1s - loss: 0.2908 - accuracy: 0.91 - ETA: 1s - loss: 0.2918 - accuracy: 0.91 - ETA: 1s - loss: 0.2892 - accura

{'drop_rate_0': 0.5,
 'drop_rate_1': 0.1,
 'drop_rate_2': 0.3,
 'drop_rate_3': 0.9,
 'num_layers': 2,
 'units_0': 56,
 'units_1': 40,
 'units_2': 32,
 'units_3': 64}


In [64]:
import kerastuner as kt
from sklearn import ensemble
from sklearn import metrics
from sklearn import datasets
from sklearn import model_selection

In [65]:
def build_tree_model(hp):      
  return ensemble.RandomForestClassifier(
      n_estimators=hp.Int('n_estimators', 10, 80, step=5),
      max_depth=hp.Int('max_depth', 3, 10, step=1),
      max_features=hp.Choice('max_features', ['auto', 'sqrt', 'log2'])
  )

sk_tuner = kt.tuners.Sklearn(
  oracle=kt.oracles.BayesianOptimization(
      objective=kt.Objective('score', 'max'),
      max_trials=MAX_TRIALS,
      seed=RANDOM_SEED
  ),
  hypermodel=build_tree_model,
  scoring=metrics.make_scorer(metrics.accuracy_score),
  cv=model_selection.StratifiedKFold(5),
  directory='test_dir',
  project_name='tune_rf'
)

In [69]:
sk_tuner.search(X_train, y_train)

In [70]:
sk_tuner.results_summary()

In [71]:
pprint(sk_tuner.oracle.get_best_trials(num_trials=1)[0].hyperparameters.values)

{'max_depth': 9, 'max_features': 'sqrt', 'n_estimators': 30}


In [ ]:
# tuner.search(X_train, y_train, epochs = 10, validation_data = (X_test, y_test), callbacks = [ClearTrainingOutput()])

# # Get the optimal hyperparameters
# best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

# print(f"""
# The hyperparameter search is complete. The optimal number of units in the first densely-connected
# layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
# is {best_hps.get('learning_rate')}.
# """)

In [ ]:
from datetime import datetime
from packaging import version
from tensorboard.plugins.hparams import api as hp

# Define the Keras TensorBoard callback.
logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
callback = keras.callbacks.TensorBoard(log_dir=logdir)
# Clear any logs from previous runs
!rm -rf ./logs/

model = tuner.hypermodel.build(best_hps)
model.fit(X_train, y_train, epochs = 10, validation_data = (X_train, y_train), callbacks=[
        tf.keras.callbacks.TensorBoard(logdir)])

In [ ]:
# %tensorboard --logdir logs

## Deploy

Save your model's weights using the Checkpoint function. Try reloading the model and making inference on your validation dataset.

### Stretch Goals
- Mount your Google Drive to Colab to persist your model checkpoint files. 
- Research L2 normalization (weight decay)
- Write a custom callback function to stop training after you reach .88 validation accuracy. 
- Select a new dataset and apply a neural network to it.
- Research TensorFlow Serving
- Play [QuickDraw](https://quickdraw.withgoogle.com/data)
- Create a static webpage using TensorFlow.js to serve a model. Check out [Teachable Machine Learning](https://teachablemachine.withgoogle.com/) for ideas. 

In [72]:
# # Configuring sweep

# hp = {
#     'method': 'random', #grid, random
#     'metric': {
#       'name': 'accuracy',
#       'goal': 'maximize'   
#     },
#     'parameters': {
#         'epochs': {
#             'values': [1, 2]
#         },
#         'batch_size': {
#             'values': [2560, 1280, 640, 500]
#         },
#         'dropout': {
#             'values': [0.3, 0.4, 0.5]
#         },
#         'conv_layer_size': {
#             'values': [16, 32, 64]
#         },
#         'weight_decay': {
#             'values': [0.0005, 0.005, 0.05]
#         },
#         'learning_rate': {
#             'values': [1e-2, 1e-3, 1e-4, 3e-4, 3e-5, 1e-5]
#         },
#         'optimizer': {
#             'values': ['adam', 'nadam', 'sgd', 'rmsprop']
#         },
#         'activation': {
#             'values': ['relu', 'elu', 'selu', 'softmax']
#         }
#     }
# }

# # Initializing new sweep
# sweep_id = wandb.sweep(sweep_config, entity="reece323", project=wandb_project)
# #training model

# def train():
#     config_defaults = {
#         'epochs': 5,     
#         'batch_size': 1280,
#         'weight_decay': 0.0005,
#         'learning_rate': 1e-3,
#         'activation': 'relu',
#         'optimizer': 'nadam',
#         'hidden_layer_size': 64,
#         'conv_layer_size': 8,
#         'dropout': 0.5,
#         'momentum': 0.9,
#         'seed': 42
#     }

#     wandb.init(config=config_defaults)

#     config = wandb.config
    
#     model = Sequential()
    
#     model.add(Conv2D(filters = config.conv_layer_size, kernel_size = (3, 3), padding = 'same', 
#                      activation ='relu', input_shape=(img_width, img_height,1)))
#     model.add(Dropout(config.dropout))

#     model.add(Conv2D(filters = config.conv_layer_size, kernel_size = (3, 3),
#                      padding = 'same', activation ='relu'))
#     model.add(MaxPooling2D(pool_size=(2, 2)))

#     model.add(Flatten())
#     model.add(Dense(config.hidden_layer_size, activation ='relu'))

#     model.add(Dense(num_classes, activation = "softmax"))

#     if config.optimizer=='sgd':
#       optimizer = SGD(lr=config.learning_rate, decay=1e-5, momentum=config.momentum, nesterov=True)
#     elif config.optimizer=='rmsprop':
#       optimizer = RMSprop(lr=config.learning_rate, decay=1e-5)
#     elif config.optimizer=='adam':
#       optimizer = Adam(lr=config.learning_rate, beta_1=0.9, beta_2=0.999, clipnorm=1.0)
#     elif config.optimizer=='nadam':
#       optimizer = Nadam(lr=config.learning_rate, beta_1=0.9, beta_2=0.999, clipnorm=1.0)

#     model.compile(loss = "categorical_crossentropy", optimizer = optimizer, metrics=['accuracy'])

#     model.fit(X_train, y_train, batch_size=config.batch_size,
#               epochs=config.epochs,
#               validation_data=(X_test, y_test),
#               callbacks=[WandbCallback(data_type="image", validation_data=(X_test, y_test), labels=class_names),
#                           EarlyStopping(patience=10, restore_best_weights=True)])
# # initializing sweep
# wandb.agent(sweep_id, train)

NameError: name 'sweep_config' is not defined